﻿Исследовательская группа GroupLens Research (http://www.grouplens.org/node/73) предлагает несколько наборов данных о рейтингах фильмов, проставленных пользователями сайта MovieLens в конце 1990-х - начале 2000-х. Наборы содержат рейтинги фильмов, метаданные о фильмах (жанр и год выхода) и демографические данные о пользователях (возраст, почтовый индекс, пол и род занятий). Такие данные часто представляют интерес для разработки систем рекомендования, основанных на алгоритмах машинного обучения. И хотя в этой книге методы машинного обучения не рассматриваются, я все же покажу, как формировать продольные и поперечные срезы таких наборов данных с целью привести их к нужному виду.
 
Набор MovieLens 1M содержит 1 миллион рейтингов 4000 фильмов, проставленных 6000 пользователей. Данные распределены по трем таблицам: рейтинги, информация о пользователях и информация о фильмах. После распаковки zip-файла каждую таблицу можно загрузить в отдельный объект DataFrame с помощью метода pandas.read_table:

In [1]:
# Путь в Windows
path = 'E:\\Disk.Yandex\\3.Программирование\\
balovstvo&checking\\python_for_data_analysis_McKinney\\pydata-book-2nd-edition\\datasets\\movielens\\'

In [7]:
# Путь в inux
path = './pydata-book-2nd-edition/datasets/movielens/'

In [3]:
import pandas as pd
import os

In [5]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip'] # Заголовки для таблицы пользователей
rnames = ['user_id', 'movie_id', 'rating', 'timestamp'] # Заголовки для таблицы оценок
mnames = ['movie_id', 'title', 'genres'] # Заголовки для таблицы фильмов 

In [8]:
users = pd.read_table(path + 'users.dat', sep='::', header=None, names=unames) # Чтение таблицы пользователей

/home/andrey/programming/miniconda3/lib/python3.4/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [10]:
rating = pd.read_table(path + 'ratings.dat', sep='::', header=None, names=rnames) # Чтение таблицы оценок

/home/andrey/programming/miniconda3/lib/python3.4/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [11]:
movies = pd.read_table(path + 'movies.dat', sep='::', header=None, names=mnames) # Чтение таблицы оценок

/home/andrey/programming/miniconda3/lib/python3.4/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


Проверяем считанные значения (первые пять строк)

In [12]:
users[:5]

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [13]:
rating[:5]

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [14]:
movies[:5]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [16]:
type(rating)

pandas.core.frame.DataFrame

Возраст и род занятий кодируются целыми числами, а расшифровка приведена в прилагаемом к набору данных файлу README. Анализ данных, хранящихся в трех таблицах, - непростая задача. Гораздо легче предварительно объединить все данные в одну таблицу. Применяя функцию merge из библиотеки pandas, сначала объединим rating с users, а затем результат объединим с movies. Pandas определяет, по каким столбцам объединять (или соединять), ориентируясь на совпадение имен:

In [17]:
data = pd.merge(pd.merge(rating, users), movies)

In [18]:
data[:5]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


In [19]:
data.ix[0]

user_id                                            1
movie_id                                        1193
rating                                             5
timestamp                                  978300760
gender                                             F
age                                                1
occupation                                        10
zip                                            48067
title         One Flew Over the Cuckoo's Nest (1975)
genres                                         Drama
Name: 0, dtype: object

Чтобы получить средние рейтинги фильмов при группировке по полу, воспользуемся методом pivot_table:

In [22]:
mean_ratings = data.pivot_table('rating', index='title', columns='gender', aggfunc='mean')

In [23]:
mean_ratings[:5]

gender,F,M
title,,
"$1,000,000 Duck (1971)",3.375000,2.761905
'Night Mother (1986),3.388889,3.352941
'Til There Was You (1997),2.675676,2.733333
"'burbs, The (1989)",2.793478,2.962085
...And Justice for All (1979),3.828571,3.689024


В результате получается еще один объект DataFrame, содержащий средние рейтинги, в котором метками строк являются общее количество оценок фильма, а метками столбцов - обозначения полов. Оставим только фильмы, получившие не менее 250 оценок; для этого сгруппируем данные по названию и с помощью метода size () полчим объект Scries, содержащий размеры групп для каждого наименования:

In [24]:
ratings_by_title = data.groupby('title').size() 

In [25]:
ratings_by_title[:10]

title
$1,000,000 Duck (1971)                37
'Night Mother (1986)                  70
'Til There Was You (1997)             52
'burbs, The (1989)                   303
...And Justice for All (1979)        199
1-900 (1994)                           2
10 Things I Hate About You (1999)    700
101 Dalmatians (1961)                565
101 Dalmatians (1996)                364
12 Angry Men (1957)                  616
dtype: int64

In [26]:
active_titles = ratings_by_title.index[ratings_by_title >= 250] 

In [27]:
active_titles

Index([''burbs, The (1989)', '10 Things I Hate About You (1999)',
       '101 Dalmatians (1961)', '101 Dalmatians (1996)', '12 Angry Men (1957)',
       '13th Warrior, The (1999)', '2 Days in the Valley (1996)',
       '20,000 Leagues Under the Sea (1954)', '2001: A Space Odyssey (1968)',
       '2010 (1984)',
       ...
       'X-Men (2000)', 'Year of Living Dangerously (1982)',
       'Yellow Submarine (1968)', 'You've Got Mail (1998)',
       'Young Frankenstein (1974)', 'Young Guns (1988)',
       'Young Guns II (1990)', 'Young Sherlock Holmes (1985)',
       'Zero Effect (1998)', 'eXistenZ (1999)'],
      dtype='object', name='title', length=1216)

Для отбора строк из приведенного выше объекта mean_ratings воспользуемся индексом фильмов, получивших не менее 250 оценок:

In [28]:
mean_ratings = mean_ratings.ix[active_titles]

In [29]:
mean_ratings

gender,F,M
title,,
"'burbs, The (1989)",2.793478,2.962085
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215
12 Angry Men (1957),4.184397,4.328421
"13th Warrior, The (1999)",3.112000,3.168000
2 Days in the Valley (1996),3.488889,3.244813
"20,000 Leagues Under the Sea (1954)",3.670103,3.709205
2001: A Space Odyssey (1968),3.825581,4.129738


Чтобы найти фильмы, оказавшиеся на первом месте у зрителей-женщин, можно отсортировать результат по столбцу F в порядке убывания:

In [31]:
top_female_ratings = mean_ratings.sort_index(by = 'F', ascending=False)

/home/andrey/programming/miniconda3/lib/python3.4/site-packages/ipykernel/__main__.py:1: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  if __name__ == '__main__':


In [33]:
top_female_ratings[:10]

gender,F,M
title,,
"Close Shave, A (1995)",4.644444,4.473795
"Wrong Trousers, The (1993)",4.588235,4.478261
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.572650,4.464589
Wallace & Gromit: The Best of Aardman Animation (1996),4.563107,4.385075
Schindler's List (1993),4.562602,4.491415
"Shawshank Redemption, The (1994)",4.539075,4.560625
"Grand Day Out, A (1992)",4.537879,4.293255
To Kill a Mockingbird (1962),4.536667,4.372611
Creature Comforts (1990),4.513889,4.272277


### Измерение несогласия в оценках

Найдем фильмы, по которым мужчины и женщины сильнее всего разошлись в оценках. Для этого можно добавить столбец mean_ratings, содержащий разность средних, а затем отсортировать по нему:

In [34]:
mean_ratings['diff'] = mean_ratings['M'] - mean_ratings['F']

In [35]:
mean_ratings

gender,F,M,diff
title,,,
"'burbs, The (1989)",2.793478,2.962085,0.168607
10 Things I Hate About You (1999),3.646552,3.311966,-0.334586
101 Dalmatians (1961),3.791444,3.500000,-0.291444
101 Dalmatians (1996),3.240000,2.911215,-0.328785
12 Angry Men (1957),4.184397,4.328421,0.144024
"13th Warrior, The (1999)",3.112000,3.168000,0.056000
2 Days in the Valley (1996),3.488889,3.244813,-0.244076
"20,000 Leagues Under the Sea (1954)",3.670103,3.709205,0.039102
2001: A Space Odyssey (1968),3.825581,4.129738,0.304156


Сортировка по столбцу diff дает фильмы с наибольшей разностью оценок, которые больше нравятся женщинам:

In [37]:
sorted_by_diff = mean_ratings.sort_index(by='diff')  

/home/andrey/programming/miniconda3/lib/python3.4/site-packages/ipykernel/__main__.py:1: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  if __name__ == '__main__':


In [38]:
sorted_by_diff[:5]

gender,F,M,diff
title,,,
Dirty Dancing (1987),3.790378,2.959596,-0.830782
Jumpin' Jack Flash (1986),3.254717,2.578358,-0.676359
Grease (1978),3.975265,3.367041,-0.608224
Little Women (1994),3.870588,3.321739,-0.548849
Steel Magnolias (1989),3.901734,3.365957,-0.535777


Изменив порядок строк на противоположный и снова отобрав первые 5 строк,
мы получим фильмы, которым мужчины поставили высокие, а женщины - низкие
оценки:

In [39]:
sorted_by_diff[::-1][:5]

gender,F,M,diff
title,,,
"Good, The Bad and The Ugly, The (1966)",3.494949,4.221300,0.726351
"Kentucky Fried Movie, The (1977)",2.878788,3.555147,0.676359
Dumb & Dumber (1994),2.697987,3.336595,0.638608
"Longest Day, The (1962)",3.411765,4.031447,0.619682
"Cable Guy, The (1996)",2.250000,2.863787,0.613787


Допустим, нас интересуют фильмы, вызвавшие наибольшее разногласие у зрителей независимо от пола. Разногласие можно изменить с помощью дисперсии или стандартного отклонения.

In [40]:
# Стандартное отклонение оценок, сгруппированных по названию
rating_std_by_title = data.groupby('title')['rating'].std()

In [41]:
# Оставляем только active_titles
rating_std_by_title = rating_std_by_title.ix[active_titles]

In [42]:
# Упорядочиваем Series по значению в порядке убывания
rating_std_by_title.order(ascending=False)[:10]

/home/andrey/programming/miniconda3/lib/python3.4/site-packages/ipykernel/__main__.py:2: FutureWarning: order is deprecated, use sort_values(...)
  from ipykernel import kernelapp as app


title
Dumb & Dumber (1994)                     1.321333
Blair Witch Project, The (1999)          1.316368
Natural Born Killers (1994)              1.307198
Tank Girl (1995)                         1.277695
Rocky Horror Picture Show, The (1975)    1.260177
Eyes Wide Shut (1999)                    1.259624
Evita (1996)                             1.253631
Billy Madison (1995)                     1.249970
Fear and Loathing in Las Vegas (1998)    1.246408
Bicentennial Man (1999)                  1.245533
Name: rating, dtype: float64